Attacks

In [1]:
#!pip install opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
import tarfile
import torch
import requests
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torchvision
import torch.utils.data as torch_data
import torchvision.transforms as transforms
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import math
#from google.colab import files

In [ ]:
## download the data
response = requests.get(f'https://www.comp.nus.edu.sg/~reza/files/dataset_texas.tgz')
if response.status_code == 200:
    with open(f'dataset_texas.tgz', 'wb') as file:
        file.write(response.content)
    print("Download completed successfully.")
else:
    print(f"Failed to download file: {response.status_code}")


In [3]:
with tarfile.open(f'dataset_texas.tgz') as f:
    f.extractall(f'data/')

with open('data/texas/100/feats', 'r') as f:
    features = f.readlines()
with open('data/texas/100/labels', 'r') as f:
    labels = f.readlines()

In [23]:
## build the sampled trainig and test datasets
features_list = [list(map(int, ''.join(feature.split()).split(','))) for feature in features]
labels_list = [int(label.strip()) for label in labels]
size = int(0.8 * len(features))
sample_size = 100
## sample dataset for inference attacks
feat_tens = torch.tensor(features_list[size-sample_size:size+sample_size], dtype=torch.float)
l_tens = torch.tensor(labels_list[size-sample_size:size+sample_size], dtype=torch.long)
dataset = TensorDataset(feat_tens, l_tens)


## true train and test datasets
feat_tens_train = torch.tensor(features_list[size-sample_size:size], dtype=torch.float)
l_tens_train = torch.tensor(labels_list[size-sample_size:size], dtype=torch.long)
#feat_tens_test = torch.tensor(features_list[size:], dtype=torch.float)
#l_tens_test = torch.tensor(labels_list[size:], dtype=torch.long)
dataset_train = TensorDataset(feat_tens_train, l_tens_train)
#dataset_test = TensorDataset(feat_tens_test, l_tens_test)

In [7]:
print(os.getcwd())
current_dir = os.getcwd()
model_path = os.path.join(current_dir,'..','Models','model_S.pth')

print(model_path)

C:\Users\ale_m\Desktop\Alles\Università\Unibas\03 Privacy-preserving methods\PrivacyProject\Membership_inference
C:\Users\ale_m\Desktop\Alles\Università\Unibas\03 Privacy-preserving methods\PrivacyProject\Membership_inference\..\Models\model_S.pth


In [9]:
## model class
class NetSeq(nn.Module):
    def __init__(self):
        super(NetSeq, self).__init__()
        self.fc1 = nn.Linear(6169, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 101)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
## load the model
model_state_dict = torch.load(model_path)
model = NetSeq()
model.load_state_dict(model_state_dict)
model.eval()

NetSeq(
  (fc1): Linear(in_features=6169, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=101, bias=True)
)

In [19]:
## evaluate the model and compute loss
criterion = nn.CrossEntropyLoss()
data_loader = DataLoader(dataset, batch_size = 1, shuffle = False)
results = []
with torch.no_grad():
    for feat, lab in data_loader:
            # Forward pass
        outputs = model(feat)    
            # Compute loss
        loss = criterion(outputs, lab)
            
            # Store the feature values and loss
        results.append((feat.numpy(), lab.numpy(), loss.item()))

## sort per loss increasingly
sorted_results = sorted(results, key = lambda x: x[2])
predicted_members = sorted_results[:sample_size]
print(len(sorted_results))
print(len(predicted_members))
##print(sorted_results[999][2])
## max loss as an upper bound for threshold in a ROC curve
max_loss = sorted_results[len(sorted_results)-1][2]
max_loss2 = predicted_members[len(predicted_members)-1][2]
int_max_loss = math.ceil(max_loss)
print(int_max_loss, max_loss2)

200
100
8 0.2831069827079773


In [25]:
## the idea is that the value for which the loss is smaller are the one that are more likely to be member of the training set
trainloader = DataLoader(dataset_train, batch_size=1, shuffle=False)

train_features_set = {tuple(features.numpy().flatten()) for features, _ in trainloader}

count_actual_members = sum(1 for feats, lab, loss in predicted_members if tuple(feats.flatten()) in train_features_set)

        
print('absolute number of correct predictions:',count_actual_members)
print('accuracy of prediction:', count_actual_members/sample_size)


absolute number of correct predictions: 53
accuracy of prediction: 0.53


In [ ]:
model = 'blabla' ## salvati come pth


x, y = 'sample of n elements'  ## check how to sample from tensor datasets
losses = 'array of losses for model(x), y'
sorted_elem = 'sort losses increasingly'

## assume that 50% of x are in the training set
## pick first n/2 x wrt to sorted loss
## check how many of these are actually in the training dataset

## in case of unknown percentage
max_loss = 'max loss'
threshold = 'between 0 and max_loss, intervall max_loss/100'

## ROc curve, for each threshold comupte how many x are chosen as members
## compute ratio TPR/FPR and plot the curve
